# 02 Transfer loop: source to target

Sections covered: 3-4 (core idea and universal loop).

**Hypothesis**: reusing source representation improves sample efficiency vs scratch.

In [ ]:
from pathlib import Path
from copy import deepcopy
import subprocess
import yaml
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path('..').resolve()
LOGS = ROOT / 'outputs' / 'logs'
FIGS = ROOT / 'outputs' / 'figures'
LOGS.mkdir(parents=True, exist_ok=True)
FIGS.mkdir(parents=True, exist_ok=True)

def run_config(config_path: Path, use_progress: bool = True):
    cmd = ['python', str(ROOT / 'scripts' / 'run_transfer.py'), '--config', str(config_path)]
    if use_progress:
        cmd.append('--use-progress')
    subprocess.run(cmd, cwd=ROOT, check=True)

def read_method(name: str) -> pd.DataFrame:
    return pd.read_csv(LOGS / f'transfer_{name}.csv')

def deep_update(base: dict, patch: dict) -> dict:
    for k, v in patch.items():
        if isinstance(v, dict) and isinstance(base.get(k), dict):
            deep_update(base[k], v)
        else:
            base[k] = v
    return base

def apply_fast_dev_profile(cfg: dict) -> dict:
    cfg = deepcopy(cfg)
    data = cfg.setdefault('data', {})
    train = cfg.setdefault('train', {})

    if 'source_train_per_class' in data:
        data['source_train_per_class'] = min(int(data['source_train_per_class']), 220)
    if 'source_test_per_class' in data:
        data['source_test_per_class'] = min(int(data['source_test_per_class']), 80)
    if 'target_train_per_class' in data:
        data['target_train_per_class'] = min(int(data['target_train_per_class']), 30)
    if 'target_test_per_class' in data:
        data['target_test_per_class'] = min(int(data['target_test_per_class']), 80)
    if 'probe_per_class' in data:
        data['probe_per_class'] = min(int(data['probe_per_class']), 40)

    data['batch_size'] = min(int(data.get('batch_size', 128)), 64)
    data['num_workers'] = 0

    if 'source_epochs' in train:
        train['source_epochs'] = min(int(train['source_epochs']), 2)
    if 'target_epochs' in train:
        train['target_epochs'] = min(int(train['target_epochs']), 3)

    train['gradual_schedule'] = {
        '1': ['backbone.layer4'],
        '2': ['backbone.layer3'],
    }
    return cfg

def make_profiled_config(base_name: str, notebook_tag: str, fast_dev_run: bool, overrides: dict | None = None) -> Path:
    cfg = yaml.safe_load((ROOT / 'configs' / base_name).read_text())
    if fast_dev_run:
        cfg = apply_fast_dev_profile(cfg)
    if overrides:
        cfg = deep_update(cfg, deepcopy(overrides))

    suffix = 'fast' if fast_dev_run else 'full'
    out = ROOT / 'configs' / f'tmp_{notebook_tag}_{suffix}.yaml'
    out.write_text(yaml.safe_dump(cfg, sort_keys=False))
    return out


In [ ]:
FAST_DEV_RUN = False
cfg_path = make_profiled_config(
    base_name='transfer_core_related.yaml',
    notebook_tag='02_core_loop',
    fast_dev_run=FAST_DEV_RUN,
)
run_config(cfg_path, use_progress=not FAST_DEV_RUN)


In [ ]:
def plot_compare(metric: str, methods):
    plt.figure(figsize=(6, 3.5))
    for m in methods:
        df = read_method(m)
        plt.plot(df['epoch'], df[metric], marker='o', label=m)
    plt.xlabel('epoch')
    plt.ylabel(metric)
    plt.legend(frameon=False)
    plt.grid(alpha=0.2)

plot_compare('target_test_acc', ['scratch', 'feature_extraction', 'gradual_unfreeze', 'naive_finetune'])

In [ ]:
summary = []
for m in ['scratch', 'feature_extraction', 'gradual_unfreeze', 'naive_finetune']:
    df = read_method(m)
    summary.append({
        'method': m,
        'best_target_acc': float(df['target_test_acc'].max()),
        'final_target_acc': float(df['target_test_acc'].iloc[-1]),
        'final_feature_drift': float(df['feature_drift'].iloc[-1]),
    })
pd.DataFrame(summary).sort_values('best_target_acc', ascending=False)


## Loop mapping

- `source pretrain` creates `theta_s`.
- each method initializes target training from either random (`scratch`) or `theta_s` (transfer methods).
- adaptation policy is the only variable changed.